In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset, load_metric
from transformers import BertTokenizer, BartTokenizer, BertModel, AutoTokenizer, AutoModel, EncoderDecoderModel, BartForConditionalGeneration
from transformers import Trainer, TrainingArguments, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import pipeline
from transformers import DataCollatorForSeq2Seq, DataCollator

# Hiding warnings
import warnings
warnings.filterwarnings("ignore")

import nltk
import tqdm

/home/chrylus/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-20 17:57:36.030376: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-20 17:57:36.061607: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
post_pre = pd.read_csv ('../Dataset/Indonesian News Dataset Preprocessed.csv', encoding='iso-8859-1')
post_pre

,text,summary
0,jakarta anggota komisi vii dpr ri rofik hanan...,_START_ anggota komisi vii dpr ri rofik hanant...
1,jakarta presiden joko widodo atau jokowi meme...,_START_ presiden joko widodo telah memerintahk...
2,wakil ketua mpr ri dr hidayat nur wahid ma at...,_START_ wakil ketua mpr ri dr hidayat nur wahi...
3,jakarta tim kedokteran dan kesehatan dokkes p...,_START_ tim kedokteran dan kesehatan dokkes po...
4,ketua mpr ri sekaligus ketua umum ikatan moto...,_START_ ketua mpr ri bambang soesatyo telah di...
...,...,...
27596,menjadi salah satu lubang terdalam dan tergela...,_START_ peneliti berhasil mengungkap sedikit d...
27597,seorang guru sd di banyuwangi ditangkap oleh p...,_START_ seorang guru sd di banyuwangi ditangka...
27598,pelatih aji santoso salut dengan sambutan saat...,_START_ pelatih persebaya aji santoso menyampa...
27599,suasana haru terjadi usai sidang terdakwa suge...,_START_ sugeng guruh gautama legiman terdakwa ...


In [3]:
# Calculate total samples
total_samples = len(post_pre)

# Calculate sizes for each split
train_size = int(0.6 * total_samples)
val_size = int(0.2 * total_samples)
test_size = total_samples - train_size - val_size

# Perform the split
train_val, test = post_pre[:train_size + val_size], post_pre[train_size + val_size:]
train, val = train_val[:train_size], train_val[train_size:]

# Verify the splits (optional)
print(f"Train samples: {len(train)}")
print(f"Validation samples: {len(val)}")
print(f"Test samples: {len(test)}")

Train samples: 16560
Validation samples: 5520
Test samples: 5521


In [4]:
# Transforming dataframes into datasets
train_ds = Dataset.from_pandas(train)
test_ds = Dataset.from_pandas(test)
val_ds = Dataset.from_pandas(val)

In [5]:
# Visualizing results
print(train_ds)
print('\n' * 2)
print(test_ds)
print('\n' * 2)
print(val_ds)

Dataset({
    features: ['text', 'summary'],
    num_rows: 16560
})



Dataset({
    features: ['text', 'summary'],
    num_rows: 5521
})



Dataset({
    features: ['text', 'summary'],
    num_rows: 5520
})


In [6]:
# Loading summarization pipeline with the bart-large-cnn model
summarizer = pipeline('summarization', model = 'facebook/bart-large-cnn')

In [7]:
checkpoint = 'facebook/bart-large-cnn' # Model
tokenizer = BartTokenizer.from_pretrained(checkpoint) # Loading Tokenizer

In [8]:
model = BartForConditionalGeneration.from_pretrained(checkpoint) # Loading Model

In [9]:
print(model) # Visualizing model's architecture

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_l

In [10]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
# Applying preprocess_function to the datasets
tokenized_train = train_ds.map(preprocess_function, batched=True,
                               remove_columns=['text', 'summary']) # Removing features

tokenized_test = test_ds.map(preprocess_function, batched=True,
                               remove_columns=['text', 'summary']) # Removing features

tokenized_val = val_ds.map(preprocess_function, batched=True,
                               remove_columns=['text', 'summary']) # Removing features

# Printing results
print('\n' * 3)
print('Preprocessed Training Dataset:\n')
print(tokenized_train)
print('\n' * 2)
print('Preprocessed Test Dataset:\n')
print(tokenized_test)
print('\n' * 2)
print('Preprocessed Validation Dataset:\n')
print(tokenized_val)

Map: 100%|██████████| 5520/5520 [00:06<00:00, 789.96 examples/s]





Preprocessed Training Dataset:

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 16560
})



Preprocessed Test Dataset:

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5521
})



Preprocessed Validation Dataset:

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5520
})


In [12]:
# Instantiating Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [13]:
metric = load_metric('rouge') # Loading ROUGE Score

In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred# Obtaining predictions and true labels
    
    # Decoding predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Obtaining the true labels tokens, while eliminating any possible masked token (i.e., label = -100)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    
    # Computing rouge score
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()} # Extracting some results

    # Add mean-generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [15]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir='bart_samsum',
    evaluation_strategy="epoch",
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    seed=42,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 2/2070 [00:34<10:15:26, 17.86s/it]

KeyboardInterrupt: 

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = 'bart_samsum',
    evaluation_strategy = "epoch",
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    metric_for_best_model = 'eval_loss',
    seed = 42,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train() # Training model

In [ ]:
# Evaluating model performance on the tokenized test dataset
validation = trainer.evaluate(eval_dataset = tokenized_test)
print(validation) # Printing results

In [ ]:
# Saving model to a custom directory
directory = "bart_finetuned_samsum_2"
trainer.save_model(directory)

# Saving model tokenizer
tokenizer.save_pretrained(directory)

In [ ]:
import shutil

In [ ]:
# Saving model in .zip format
shutil.make_archive('bart_finetuned_samsum', 'zip', '/kaggle/working/bart_finetuned_samsum')
shutil.move('bart_finetuned_samsum.zip', '/kaggle/working/bart_finetuned_samsum.zip')

In [ ]:
pipe = pipeline('summarization', model=directory)

In [ ]:
text = "perempuan berinisial ag ditahan oleh penyidik ditreskrimum polda metro jaya usai enam jam diperiksa di kasus penganiayaan cristalino david ozora dari hasil pemeriksaan kami selama kurang lebih enam jam kami sekali lagi dengan pertimbangan kenyamanan terhadap anak malam ini kami putuskan dari penyidik kemudian untuk melakukan penangkapan dan dilanjutkan dengan penahan kata direktur reskrimum polda metro jaya kombes hengki haryadi dalam konferensi pers rabu 8/3 hengki turut menjelaskan selama enam jam pemeriksaan itu pihaknya memastikan telah memenuhi seluruh hak ag selaku anak pemenuhan hak anak dari ag kata dia sesuai dengan ketentuan yang diatur dalam sistem peradilan anak dalam hal ini didampingi selain daripada lawyer juga tadi dari tim pembimbing kemasyarakatan bapas jaksel dan juga untuk menjamin pemenuhan hak anak juga didampingi tim dari kemenpppa yang juga merangkap pendamping psikososial dalam rangka dan dalam menjamin pemenuhan terhadap hak hak anak tuturnya sebelumnya ag yang berstatus sebagai pelaku atau anak yang berkonflik dengan hukum resmi ditahan sejak rabu 8/3 kemarin ag bakal ditahan selama tujuh hari ke depan ag dijerat pasal 76c jo pasal 80 uu ppa dan atau pasal 355 ayat jo pasal 56 kuhp subsider pasal 354 ayat jo 56 kuhp subsider 353 ayat jo pasal 56 kuhp mario dijerat dengan pasal 355 kuhp ayat subsider pasal 354 ayat kuhp subsider 535 ayat kuhp subsider 351 ayat kuhp penyidik juga mengenakan mario pasal 76c jo 80 undang-undang perlindungan anak sementara shane dijerat pasal 355 ayat jo pasal 56 kuhp subsider 354 ayat jo 56 kuhp subsider 353 ayat jo 56 kuhp subsider 351 ayat jo 76c undang-undang perlindungan anak."
summary = "ag ditahan oleh penyidik polda metro jaya usai diperiksa selama enam jam dalam kasus penganiayaan cristalino david ozora penahanan dilakukan dengan konsiderasi kenyamanan ag sebagai anak ag dijerat pasal 76c jo pasal 80 uu ppa dan atau pasal 355 ayat jo pasal 56 kuhp subsider pasal 354 ayat jo 56 kuhp subsider 353 ayat jo pasal 56 kuhp ag bakal ditahan selama tujuh hari ke depan."
generated_summary = pipe(text)

In [ ]:
print('Original Dialogue:\n')
print(text)
print('\n' * 2)
print('Reference Summary:\n')
print(summary)
print('\n' * 2)
print('Model-generated Summary:\n')
print(generated_summary)

In [ ]:
text2 = "jakarta cnbc indonesia ceo silicon valley bank svb atau bank terbesar di amerika serikat as yang kolaps greg becker tidak lagi menjadi dewan direksi di federal reserve bank of san francisco juru bicara federal reserve menyebut kepergian becker efektif pada jumat 10/3/2023 pada hari yang sama perusahaan perbankan komersial berbasis california amerika serikat as svb mengalami krisis modal dan bangkrut dalam 48 jam terakhir svb kemudian ditutup oleh regulator selain menutup svb regulator keuangan juga mengambil kendali atas depositonya hal ini diumumkan oleh federal deposit insurance corp fdic melansir reuters kuru bicara itu menolak mengatakan bagaimana becker keluar dari dewan fed san francisco becker menjabat sebagai direktur kelas di san francisco fed salah satu dari tiga eksekutif keuangan yang mewakili bank anggota di distrik fed san francisco setiap bank daerah diawasi oleh dewan yang terdiri dari warga negara selain memiliki tiga direktur yang mewakili bank ada enam direktur lainnya yang menghadirkan perpaduan antara bisnis lokal dan kepentingan masyarakat tiga dari direktur tersebut dipilih oleh dewan gubernur fed di washington sedangkan sisanya dipilih dalam proses lokal sebanyak 12 bank federal reserve regional adalah lembaga kuasi-swasta yang diawasi oleh the fed di washington dewan masing-masing mengawasi bank secara langsung dan memberikan nasihat tentang tata kelola serta kecerdasan ekonomi lokal dewan ini juga memimpin proses pemilihan presiden baru ketika ada lowongan meskipun direktur dari perusahaan yang diatur oleh fed tidak diizinkan untuk berpartisipasi dalam proses tersebut para direktur bank fed telah menjadi sorotan dalam beberapa tahun terakhir karena bank sentral menghadapi kritik terkait direktur bank tidak memiliki keragaman ras dan gender serta terlalu membebani komunitas bisnis dan perbankan the fed telah bekerja untuk memperluas siapa yang melayani dalam peran ini."
summary2 = "ceo svb bank terbesar di as greg becker mundur dari dewan direksi federal reserve bank of san francisco hal ini terkait dengan krisis modal dan bangkrutnya svb dalam 48 jam terakhir serta pengambilalihan deposito oleh regulator meskipun dewan gubernur fed di washington yang memimpin pemilihan presiden baru direktur dari perusahaan yang diatur oleh fed tidak diizinkan untuk berpartisipasi dalam proses itu penunjukan direktur bank fed juga telah menghadapi kritik terkait dengan tidak memiliki keragaman ras dan gender."
generated_summary2 = pipe(text2)

In [ ]:
print('Original Dialogue:\n')
print(text2)
print('\n' * 2)
print('Reference Summary:\n')
print(summary2)
print('\n' * 2)
print('Model-generated Summary:\n')
print(generated_summary2)